In [ ]:
import pandas as pd
import numpy as np
import pm4py
import graphviz
from sklearn.preprocessing import MinMaxScaler
from keras_preprocessing.sequence import pad_sequences
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Batch
#%config Completer.use_jedi = False
#"python.dataScience.enablePlotViewer": false
import networkx as nx 
import torch
from torch_geometric.data import Data
from tqdm import  tqdm
import warnings
import plotly.express as px
import datetime
from sklearn.preprocessing import OneHotEncoder
import networkx as nx
from torch_geometric.utils.convert import to_networkx
warnings.filterwarnings('ignore')

### Creating two different Tables

In [ ]:
file_path = "Event_log.csv"
df = pd.read_csv(file_path, sep=";")
df['TIMESTAMP']= pd.to_datetime(df['TIMESTAMP'])
df["YEAR"] = df["TIMESTAMP"].dt.year
df.head()

,CASE_ID,ACTIVITY,TIMESTAMP,REPAIR_IN_TIME_5D,DEVICETYPE,SERVICEPOINT,YEAR
0,Case10,Creation,2022-01-02 13:39:47,0,AB52,E,2022
1,Case10,Letter,2022-01-05 00:00:00,0,AB52,E,2022
2,Case10,DeviceReceived,2022-01-05 16:45:34,0,AB52,E,2022
3,Case10,StockEntry,2022-01-17 00:00:00,0,AB52,E,2022
4,Case10,InDelivery,2022-01-17 00:00:00,0,AB52,E,2022


In [ ]:
a = df.groupby(["CASE_ID", "SERVICEPOINT"])#.agg({"SERVICEPOINT" : pd.unique})
a.head(200)

,CASE_ID,ACTIVITY,TIMESTAMP,REPAIR_IN_TIME_5D,DEVICETYPE,SERVICEPOINT,YEAR
0,Case10,Creation,2022-01-02 13:39:47,0,AB52,E,2022
1,Case10,Letter,2022-01-05 00:00:00,0,AB52,E,2022
2,Case10,DeviceReceived,2022-01-05 16:45:34,0,AB52,E,2022
3,Case10,StockEntry,2022-01-17 00:00:00,0,AB52,E,2022
4,Case10,InDelivery,2022-01-17 00:00:00,0,AB52,E,2022
...,...,...,...,...,...,...,...
178073,Case9999,DeviceReceived,2022-06-22 13:57:52,1,AB34,L,2022
178074,Case9999,Letter,2022-06-25 00:00:00,1,AB34,L,2022
178075,Case9999,InDelivery,2022-06-26 00:00:00,1,AB34,L,2022
178076,Case9999,NoteWorkshop,2022-06-26 10:48:00,1,AB34,L,2022


In [ ]:
check_nan = df[df["SERVICEPOINT"].isna()]
print(check_nan.isna().sum())
l = check_nan.CASE_ID.unique()
print(len(l))

print(df[df["CASE_ID"].isin(l)].isna().sum())
print(df[df["CASE_ID"].isin(l)].info())
print(len(l))

CASE_ID                  0
ACTIVITY                 0
TIMESTAMP                0
REPAIR_IN_TIME_5D        0
DEVICETYPE              13
SERVICEPOINT         16518
YEAR                     0
dtype: int64
1975
CASE_ID                  0
ACTIVITY                 0
TIMESTAMP                0
REPAIR_IN_TIME_5D        0
DEVICETYPE              13
SERVICEPOINT         16518
YEAR                     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16524 entries, 124 to 178024
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CASE_ID            16524 non-null  object        
 1   ACTIVITY           16524 non-null  object        
 2   TIMESTAMP          16524 non-null  datetime64[ns]
 3   REPAIR_IN_TIME_5D  16524 non-null  int64         
 4   DEVICETYPE         16511 non-null  object        
 5   SERVICEPOINT       6 non-null      object        
 6   YEAR               16524 non-null  

In [ ]:
activity_table = df[["CASE_ID",	"ACTIVITY",	"TIMESTAMP"]]
def ceation_activity(x):
    
    if x.isnull().values.any():
        res = np.NaN
        a = x.dropna()
        if len(a) > 0:
            res = a.values[0]

        return res
    return x.iloc[0]

case_table = df.groupby("CASE_ID").agg(
    start_time = ('TIMESTAMP', min),
    end = ('TIMESTAMP', max),
    duration = ("TIMESTAMP", lambda x: (max(x) - min(x))),
    REPAIR_IN_TIME_5D = ("REPAIR_IN_TIME_5D", max),
    DEVICETYPE = ("DEVICETYPE", lambda x: ceation_activity(x)),
    SERVICEPOINT = ("SERVICEPOINT", lambda x: ceation_activity(x)),
)

In [ ]:
file_path = "Event_log.csv"
df = pd.read_csv(file_path, sep=";")
df['TIMESTAMP']= pd.to_datetime(df['TIMESTAMP'])
df["YEAR"] = df["TIMESTAMP"].dt.year
df.head()

unique_case_ids = df.CASE_ID.unique()

random_lit_cse_ids = np.random.choice(unique_case_ids, size=500, replace=False)
sampled_df = df[df["CASE_ID"].isin(random_lit_cse_ids)  ]

In [ ]:
class Prefixing():
    def __init__(self, case_id_col:str, activity_col:str, time_col:str, general_columns:list,event_columns:list,target_columns:list, cat_columns:list, cat_encoder_type,cat_col_unique_dict:dict, start_event:str = None, end_event:str= None) -> None:
        
        self.case_id_col            = case_id_col
        self.activity_col           = activity_col
        self.time_col               = time_col
        self.start_event            = start_event
        self.end_event              = end_event
        self.general_columns        = general_columns
        self.cat_columns            = cat_columns
        self.event_columns          = event_columns
        self.target_columns         = target_columns
        self.cat_encoder_type       = cat_encoder_type
        self.encoder_dict           = {}
        self.cat_col_unique_dict    = cat_col_unique_dict
        self.activity_index_dict_for_caseID = None
        self.set_encoder()
    def get_encoder(self):
        return self.encoder_dict
    
    def get_activity_index_dict_for_caseID(self):
        return self.activity_index_dict_for_caseID
    def set_encoder(self):
        
        for cat_col in self.cat_columns:
            values = np.array(self.cat_col_unique_dict[cat_col]).reshape(-1,1)
            enc = self.cat_encoder_type(handle_unknown='ignore')
            enc.fit(values)
            self.encoder_dict[cat_col] = enc
        return None
    def check_prefix_list(self,number_activities_case):
        #print(self.prefix_list)
        return [value for value in self.prefix_list if value < number_activities_case]
    def get_caseID_dict(self, case_id_df_array):
        activity_index_dict_for_caseID = {}

        for ind in list(range(0,case_id_df_array.shape[0])):
            v = case_id_df_array[ind,1]
            if v not in list(activity_index_dict_for_caseID.keys()):
                activity_index_dict_for_caseID[v] = ind

        return activity_index_dict_for_caseID
    def get_edge_index_list(self, activity_index_dict_for_caseID,prefix_length_array):
        actions_as_int = np.array([activity_index_dict_for_caseID[action] for action in prefix_length_array[:,1]])
        

        actions_edges = actions_as_int[1:]
        actions_as_int = actions_as_int[:-1]
        
        return np.vstack((actions_as_int, actions_edges))
    
    def get_general_feature(self,process_df):
        return_array = None
        n_f = 0
        #print(list(process_df.columns))
        for pd_col in list(process_df.columns):
            f_values = np.array([process_df[pd_col].to_numpy().reshape(-1,1)[0]])
            if pd_col in self.cat_columns:
                f_values = self.encoder_dict[pd_col].transform(f_values).toarray()
            if n_f > 0:
                return_array = np.hstack((return_array , f_values))
            else: return_array = f_values
            n_f += 1

        #print(f'GeneraltFeatureArray:.shape{return_array.shape}')
        return torch.from_numpy(return_array).float()
    def get_event_features(self,event_df):
        return_array = None
        n_f = 0
        #print(list(process_df.columns))
        for pd_col in list(event_df.columns):
            f_values = event_df[pd_col].to_numpy().reshape(-1,1)
            if pd_col in self.cat_columns:
                f_values = self.encoder_dict[pd_col].transform(f_values).toarray()
            if n_f > 0:
                return_array = np.hstack((return_array , f_values))
            else: return_array = f_values
            n_f += 1

        #print(f'EventFeaturesArray:.shape{return_array.shape}')

        return torch.from_numpy(return_array)
    def transform(self, df:pd.DataFrame,prefix_list :list):
        self.prefix_list = prefix_list
        graph_list = []
        df.sort_values(by=self.time_col, inplace = True)
        for case_ind, unique_id in enumerate(tqdm(df[self.case_id_col].unique())):

            case_id_df = df[df[self.case_id_col] == unique_id]
            event_df = case_id_df[self.event_columns]
            process_df = case_id_df[self.general_columns]
            process_features = self.get_general_feature(process_df)
            event_features = self.get_event_features(event_df)

            case_id_df_array = case_id_df.to_numpy()
            number_activities_case = len(case_id_df)
            prefix_list = self.check_prefix_list(number_activities_case)
            activity_index_dict_for_caseID = self.get_caseID_dict(case_id_df_array)
            self.activity_index_dict_for_caseID = activity_index_dict_for_caseID
            edge_index_list = self.get_edge_index_list(activity_index_dict_for_caseID,case_id_df_array )
            y = torch.from_numpy(case_id_df[self.target_columns].to_numpy().astype(int))[0]
            for pref_ind , prefix_length in enumerate(prefix_list):
                edge_index = torch.from_numpy(edge_index_list[:,0:-(number_activities_case-prefix_length)])
                x = event_features[0:-(number_activities_case-prefix_length),:]
                graph = Data()
                graph.x = x.float()
                graph.general_f = process_features
                graph.edge_index = edge_index
                graph.y = y
                graph_list.append(graph)

        print(f'Number of Graphs: {len(graph_list)} from  {len(df[self.case_id_col].unique())} Cases')
        return graph_list

In [ ]:
case_id_col = "CASE_ID"
activity_col = "ACTIVITY"
time_col = "TIMESTAMP"
cat_col_unique_dict = {"SERVICEPOINT": df.SERVICEPOINT.unique(), "DEVICETYPE": df.DEVICETYPE.unique(), "ACTIVITY": df.ACTIVITY.unique()}
target_columns = ["REPAIR_IN_TIME_5D"]
event_columns = ["ACTIVITY"]
general_columns = ["SERVICEPOINT","DEVICETYPE"]
cat_columns = ["SERVICEPOINT", "DEVICETYPE", "ACTIVITY"]


### Train und Test Split -> Data Generation

In [ ]:
def train_test(df, train_ratio):#shuffle
    train_size = int(df.CASE_ID.nunique() * train_ratio)
    print(f'Gesamt: {df.CASE_ID.nunique()} Cases')
    print(f'Ratio {train_ratio}')
    print(f'train_size: {train_size} Cases')
    
    unique_case_ids = df.CASE_ID.unique()
    random_lit_cse_ids = np.random.choice(unique_case_ids, size=train_size, replace=False)
    train_df = df[df["CASE_ID"].isin(random_lit_cse_ids)  ]
    test_df = df[~df["CASE_ID"].isin(random_lit_cse_ids)  ]

    return train_df, test_df

#train_df, test_df = train_test(df,0.5)
train_df, test_df = train_test(df,0.9)

Gesamt: 23906 Cases
Ratio 0.9
train_size: 21515 Cases


In [ ]:
# Batch for Test
prefix = Prefixing(case_id_col , activity_col,time_col, general_columns,event_columns, target_columns,cat_columns, OneHotEncoder,cat_col_unique_dict )
test_graph_list =  prefix.transform(test_df, [2,4,5,8])
test_batch = Batch().from_data_list(test_graph_list)

  0%|          | 0/2391 [00:00<?, ?it/s]

100%|██████████| 2391/2391 [00:13<00:00, 182.94it/s]


Number of Graphs: 7430 from  2391 Cases


In [ ]:
# For the Training
def generate_batches(graph_data, batch_size):#shuffledd
    len_data = len(graph_data)
    rest = int(len_data%batch_size)
    number_batches= int((len_data-rest) / batch_size) 
    rest_batch = []
    if rest >0:
        batches = graph_data[0:-rest:1] 
        rest_batch = [graph_data[-rest::1]]
    else: 
        batches = graph_data 
    batches_list = []
    for n in range(0,number_batches):
        start = n * batch_size
        end = start + batch_size

        batch = batches[start:end:1]
        batches_list.append(batch)
    
    batches_list += rest_batch
    
    # rest_batch = Drop the last values in own list
    # batches = reshape the remaining into the shape (number_batches X batch_size)
    # Append rest_batches to batches 

    print(f'len data: {len_data}')
    print(f'rest: {rest}')
    print(f'number_batches: {number_batches}')
    print(f'rest_batch: {rest_batch}')
    print(f'batches: {len(batches)}')
    batches_list_l = []
    for bat in batches_list:
        batches_list_l.append(Batch().from_data_list(bat))
    return batches_list_l

#train_batches = generate_batches(res, 4)


batches_list = generate_batches(prefix.transform(df, [2,4,5,8]), 64)



print(len(batches_list))

  0%|          | 0/23906 [00:00<?, ?it/s]

100%|██████████| 23906/23906 [09:24<00:00, 42.37it/s]


Number of Graphs: 74611 from  23906 Cases
len data: 74611
rest: 51
number_batches: 1165
rest_batch: [[Data(x=[2, 13], general_f=[1, 77], edge_index=[2, 1], y=[1]), Data(x=[4, 13], general_f=[1, 77], edge_index=[2, 3], y=[1]), Data(x=[5, 13], general_f=[1, 77], edge_index=[2, 4], y=[1]), Data(x=[2, 13], general_f=[1, 77], edge_index=[2, 1], y=[1]), Data(x=[4, 13], general_f=[1, 77], edge_index=[2, 3], y=[1]), Data(x=[5, 13], general_f=[1, 77], edge_index=[2, 4], y=[1]), Data(x=[2, 13], general_f=[1, 77], edge_index=[2, 1], y=[1]), Data(x=[4, 13], general_f=[1, 77], edge_index=[2, 3], y=[1]), Data(x=[5, 13], general_f=[1, 77], edge_index=[2, 4], y=[1]), Data(x=[8, 13], general_f=[1, 77], edge_index=[2, 7], y=[1]), Data(x=[2, 13], general_f=[1, 77], edge_index=[2, 1], y=[1]), Data(x=[4, 13], general_f=[1, 77], edge_index=[2, 3], y=[1]), Data(x=[5, 13], general_f=[1, 77], edge_index=[2, 4], y=[1]), Data(x=[2, 13], general_f=[1, 77], edge_index=[2, 1], y=[1]), Data(x=[4, 13], general_f=[1, 

### Model

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(13, 8)
        self.conv2 = GCNConv(8, 8)
        self.lin = Linear(8, 8)
        self.lin_2 = Linear(8, 1)
    def forward(self, data):
        x, edge_index,batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        
        #x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index)
        #print(f'x.shape {x.shape}')
        x = global_mean_pool(x, batch=batch)
        #print(f'global_mean_pool x.shape {x.shape}')
        x = self.lin(x)
        x = self.lin_2(F.relu(x))
        return torch.sigmoid(x)

In [ ]:
class GCNEmbedding(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.gen_lin1 = Linear(77, 16)
        self.gen_lin2 = Linear(16, 8)
    def forward(self, data):
        x, edge_index,batch, general_data = data.x, data.edge_index, data.batch, data.general_f
        general_data = self.gen_lin1(general_data)
        general_data = F.relu(general_data)
        general_data = self.gen_lin2(general_data)
        general_data = F.relu(general_data)
        return general_data

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(13, 8)
        self.conv2 = GCNConv(8, 8)
        self.lin = Linear(8, 8)
        self.lin_2 = Linear(8, 1)
        self.embedding_layer = GCNEmbedding()
    def forward(self, data):
        x, edge_index,batch, general_data = data.x, data.edge_index, data.batch, data.general_f
        general_data = self.embedding_layer(data)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch=batch)
        x = x + general_data
        x = self.lin(x)
        x = self.lin_2(F.relu(x))
        return torch.sigmoid(x)
    

gcn = GCN()
g = batches_list[0]
print(g)
#gcn(g)

### Training & Testing

#### Training

In [ ]:
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss() #torch.nn.CrossEntropyLoss()
#batches_list

def train():
    model.train()
    score = 0
    for data in batches_list:

            out = model(data)  
            loss = criterion(out, torch.reshape(data.y.float(), (-1,1)))  # Compute the loss.
            score += loss
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.
            optimizer.zero_grad()  # Clear gradients.

    model.eval()

    return None

def test():
     model.eval()
     test_out = model(test_batch) 
     loss = criterion(test_out, torch.reshape(test_batch.y.float(), (-1,1)))
     return loss

t_l = test()
print(f' Test Loss before training = {t_l}')
for epoch in range(1, 10):
    train()
    
    if epoch % 5 == 0:
        t_l = test()
        print(f' Test Loss after epoche: {epoch} = {t_l}')

 Test Loss before training = 0.6753789186477661
 Test Loss after epoche: 5 = 0.43117618560791016
